# Finding a Shared Spliced Motif

Link: https://rosalind.info/problems/lcsq/

In [47]:
//:dep polars = { version = "0.32.0", features = ["describe", "lazy", "ndarray", "object", "dtype-struct", "concat_str", "mode", "polars-lazy"] }
:dep ndarray = { version = "0.15.6" }

In [95]:
use std::fs::File;
use std::io::{BufReader, BufRead};
use std::collections::HashMap;
use ndarray::prelude::*
use std::cmp;
//use polars::prelude::*;

In [96]:
let mut a = Array::<f64, _>::zeros((3, 2, 4));

In [97]:
a[(0,1, 0)] = 12.0

()

In [98]:
a

[[[0.0, 0.0, 0.0, 0.0],
  [12.0, 0.0, 0.0, 0.0]],

 [[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0]],

 [[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0]]], shape=[3, 2, 4], strides=[8, 4, 1], layout=Cc (0x5), const ndim=3

In [99]:
#[derive(Debug, Clone)] 
struct DNASeq {
    seq: String,
}

impl DNASeq {
    
    fn len(&self) -> usize {
        self.seq.len()
    }
    
    fn push_str(&mut self, s: &str) {
        self.seq.push_str(s);
    }
    
    fn subsequence_matrix(&self, other: &Self) {
        let mut seq_1 = self.seq.clone();
        seq_1.insert(0, '0');
        let mut seq_2 = other.seq.clone();
        seq_2.insert(0, '0');
        let vec_for_array_initialization = vec![0; seq_1.len() * seq_2.len()];
        let mut matrix = Array::from_shape_vec((seq_1.len(), seq_2.len()), vec_for_array_initialization).unwrap();
        for (i, x) in seq_1.chars().enumerate() {
            for (j, y) in seq_2.chars().enumerate() {
                if (x != '0') & (y != '0') {
                    match x {
                        y => matrix[(i, j)] = matrix[(i-1, j-1)] + 1,
                        _ => matrix[(i, j)] = cmp::max(matrix[(i-1, j)], matrix[(i, j-1)]),
                    }
                }
            }
        }
       println!("{:?}", matrix);  
    }
    
}

In [100]:
fn read_fasta(file_path: &str) -> HashMap<String, DNASeq> {
    let mut data = HashMap::new();
    let file = File::open(file_path).expect("Invalid filepath");
    let reader = BufReader::new(file);
    
    let mut seq_id = String::new();
    for line in reader.lines() {
        let line = line.unwrap();
        if line.starts_with('>') {
            seq_id = line.trim_start_matches('>').to_string();
        } else {
            data.entry(seq_id.clone()).or_insert(DNASeq {seq: "".to_string() }).push_str(&line);
        }
    }
    
    data
}

In [101]:
let data: HashMap<String, DNASeq> = read_fasta("data/rosalind_lcsq.txt");
let data: Vec<DNASeq> = data.into_values().collect();

In [102]:
data[0].subsequence_matrix(&data[1])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 2, 2, 2, 2, 2, 2, 2, 2],
 [0, 1, 2, 3, 3, 3, 3, 3, 3, 3],
 [0, 1, 2, 3, 4, 4, 4, 4, 4, 4],
 [0, 1, 2, 3, 4, 5, 5, 5, 5, 5],
 [0, 1, 2, 3, 4, 5, 6, 6, 6, 6],
 [0, 1, 2, 3, 4, 5, 6, 7, 7, 7],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 8]], shape=[9, 10], strides=[10, 1], layout=Cc (0x5), const ndim=2


()